In [24]:
import numpy as np
import math
from qiskit.circuit import QuantumCircuit,QuantumRegister
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
# from AdaptVQE_.My_adapt import QubitAdapt,MyAdaptVQE
from AdaptVQE_.MyQubit_Adapt_VQE import QubitAdapt
from AdaptVQE_.My_adapt import MyAdaptVQE
# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [ ]:
from qiskit.quantum_info import SparsePauliOp
#a = MyAdaptVQE(ES_problem=problem)
b = QubitAdapt(ES_problem=problem)


In [ ]:
X =b.run_slover()

In [ ]:
X[0].optimal_circuit.decompose(reps=2).draw()
X[2].optimal_point #0.10284049810578781
#

In [30]:
problem.nuclear_repulsion_energy #1.98441454095
X[2].optimal_value #-9.614201456127459
X[2].optimal_value + problem.nuclear_repulsion_energy

-7.629786915177458

In [27]:
qc = X[2].optimal_circuit
qc.bind_parameters(X[2].optimal_point).draw()
X[2].optimal_circuit.depth()

4

In [ ]:
from qiskit.quantum_info import Statevector
qc = QuantumCircuit(b.n_qubit)
qc.append(a.excitation_pool_instruction[1],range(b.n_qubit))
Statevector(data=qc.bind_parameters([0.0]))
vqe = VQE(ansatz=qc,estimator=Estimator(),optimizer=SLSQP())
vqe.compute_minimum_eigenvalue(b.hamiltonian)


In [ ]:
from qiskit.primitives.utils import bound_circuit_to_instruction
bound_circuit_to_instruction(circuit=qc)

In [ ]:
b.paulistring_withoutz_instruction[2]

In [ ]:
b.run()

In [ ]:
qc = QuantumCircuit(b.n_qubit)
qc.append(b.init_state_hf,range(b.n_qubit))
# qc.append(b.paulistring_withoutz_instruction[1],range(b.n_qubit))
qc.append(a.excitation_pool_instruction[2],range(b.n_qubit))
vqe = VQE(estimator=Estimator(),ansatz=qc,optimizer=SLSQP(),initial_point=np.zeros(qc.num_parameters))



In [ ]:
vqe.compute_minimum_eigenvalue(b.hamiltonian)

In [ ]:
from AdaptVQE_.My_adapt import MyAdaptVQE
a = MyAdaptVQE(ES_problem=problem)
a.run_tmp()

In [ ]:
#a.slover.ansatz.append(b.paulistring_withoutz_instruction[3],range(4))
a.slover.initial_point=np.zeros(2)
a.slover.compute_minimum_eigenvalue(a.hamiltonian)

In [ ]:
 def evaluate_energy(parameters: np.ndarray) -> np.ndarray | float:
            nonlocal eval_count

            # handle broadcasting: ensure parameters is of shape [array, array, ...]
            parameters = np.reshape(parameters, (-1, num_parameters)).tolist()
            batch_size = len(parameters)

            try:
                job = self.estimator.run(batch_size * [ansatz], batch_size * [operator], parameters)
                estimator_result = job.result()
            except Exception as exc:
                raise AlgorithmError("The primitive job to evaluate the energy failed!") from exc

            values = estimator_result.values

            if self.callback is not None:
                metadata = estimator_result.metadata
                for params, value, meta in zip(parameters, values, metadata):
                    eval_count += 1
                    self.callback(eval_count, params, value, meta)

            energy = values[0] if len(values) == 1 else values

            return energy

        return evaluate_energy

In [ ]:
a = None
a == None

In [ ]:
from qiskit.quantum_info import Statevector
Statevector(bound_circuit_to_instruction(circ))